# Running Dask on AzureML

This notebook shows how to run a Dask cluster on an AzureML Compute cluster. This notebook is designed to run on an AzureML Notebook VM, but it should work on your local computer, too. The changes to nginx, however, are only required on the notebook VM.

## Patching nginx configuration on the Notebook VM
To make sure you can monitor your dask cluster, you need to make a change to your notebook VM (this will no longer be required once [bug 443670](https://msdata.visualstudio.com/Vienna/_workitems/edit/443670) is fixed).

Open a terminal window on the notebook VM and do the following:

1. open `/etc/nginx/nginx.conf` in an editor
2. find this line  
 `location ~ (/api/kernels/|/terminals/websocket/) {`  
 and add `|ws`, so it looks like this  
 `location ~ (/api/kernels/|/terminals/websocket/|/ws) {`  
3. run `sudo systemctl reload nginx`

Unfortunately, you need to rerun this each time you restart the notebook VM -- again, until we fix [bug 443670](https://msdata.visualstudio.com/Vienna/_workitems/edit/443670).


## Python Environment
The environment you are running should have the latest version of `dask` and `distributed` installed -- run this code in this notebook to make sure.

    import sys
    !{sys.executable[:-6]}/pip install --upgrade dask distributed


Or, if you want to be on the safe side, create a new conda environment using this [environment.yml](dask/environment.yml) file like so:

    conda env create -f dask/environment.yml  
    conda activate dask
    python -m ipykernel install --user --name dask --display-name "Python (dask)"

## Starting the cluster

In [210]:
from azureml.core import Workspace, Experiment
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from azureml.core.runconfig import MpiConfiguration
from azureml.core import VERSION
VERSION

'1.0.41'

Now we will get the workspace and AML compute cluster and start the Dask cluster on it. The assumption is that you have created a cluster with the name `dask` -- else change the name below accordingly. **It is important that, as you created the cluster, you have provided a username (I am using `daskuser`) and password or ssh key (I am using my ssh key), since you will need to log in to the worker nodes to establish the port forwarding to the docker container.**

![create_cluster](img/create_cluster.png)

In [211]:
ws = Workspace.from_config()

In [164]:
dask_cluster = ws.compute_targets['dask']

Starting the Dask cluster using an Estimator with MpiConfiguration. Make sure the cluster is able to scale up to 10 nodes or change the `node_count` below. Also, this example is launching 2 workers per node (assuming 2 core machines). If you are running on more cores, you can change the `process_count_per_node` below.

In [228]:
mpi_configuration = MpiConfiguration()
mpi_configuration.process_count_per_node = 2

est = Estimator('dask', 
                compute_target=dask_cluster, 
                entry_script='startDask.py', 
                conda_dependencies_file_path='environment.yml', 
                node_count=10,
                distributed_training=mpi_configuration)

run = Experiment(ws, 'dask').submit(est)

In [229]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [235]:
while not 'headnode' in run.get_metrics():
    print("waiting for scheduler node's ip")
    time.sleep(5)

print('Headnode has IP:', run.get_metrics()['headnode'])

Headnode has IP: 10.0.0.4


## Establish the port-forwarding from Notebook VM to Dask Scheduler
Since Notebook VM does not yet support VNets, you need to build an SSH port forwarder through SSH login.

First you need to find the IP and port of one of the nodes on the cluster by going to the nodes tab of the cluster. Note down the IP and port -- in my case it is `52.137.63.123` and port `50000`.

![compute_nodes](img/compute_nodes.png)

Then, open the terminal on the Notebook VM and type the following:  
`ssh daskuser@<clusternode IP> -p <clusternode port> -L 8786:<headnode IP>:8786 -L 8787:<headnode IP>:8787`

In my case it is:
`ssh daskuser@52.137.63.123 -p 50000 -L 8786:10.0.0.4:8786 -L 8787:10.0.0.4:8787`

Make sure to leave the terminal tab open to keep the port-forward running

As you see, you are forwarding 2 ports 8786 is for the scheduler and 8787 is for the Bokeh app that shows the activity on the cluster. To access the Bokeh app, change the URL to your notebook VM by adding `-8787` right after the machine name. In my case it looks like this:

https://danielsctest-8787.westeurope.notebooks.azureml.net/

Hopefully, you are seeing this after you clicked on 'Status':

![Bokeh](img/bokeh.png)

## Run some jobs on the cluster
If you are able to see the Bokeh app, it is time to use the cluster. Thanks to the port forward, the scheduler appears to the notebook VM at `tcp://localhost:8786`. You should see 19 workers.

In [237]:
from dask.distributed import Client

c = Client('tcp://localhost:8786')
c

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 19 Cores: 38 Memory: 138.86 GB


See if the cluster works

In [240]:
def inc(x):
    return x + 1

fut = c.submit(inc, 1)
fut

<Future: status: pending, key: inc-c1742b26b57dbff12ab2ac87615a2cd8>

In [241]:
fut.result()

2

# Training on Large Datasets
(from https://github.com/dask/dask-tutorial)

Sometimes you'll want to train on a larger than memory dataset. `dask-ml` has implemented estimators that work well on dask arrays and dataframes that may be larger than your machine's RAM.

In [242]:
from dask.distributed import Client
import joblib
import dask.array as da
import dask.delayed
from sklearn.datasets import make_blobs
import numpy as np

We'll make a small (random) dataset locally using scikit-learn.

In [243]:
n_centers = 12
n_features = 20

X_small, y_small = make_blobs(n_samples=1000, centers=n_centers, n_features=n_features, random_state=0)

centers = np.zeros((n_centers, n_features))

for i in range(n_centers):
    centers[i] = X_small[y_small == i].mean(0)
    
centers[:4]

array([[ 1.00796679,  4.34582168,  2.15175661,  1.04337835, -1.82115164,
         2.81149666, -1.18757701,  7.74628882,  9.36761449, -2.20570731,
         5.71142324,  0.41084221,  1.34168817,  8.4568751 , -8.59042755,
        -8.35194302, -9.55383028,  6.68605157,  5.34481483,  7.35044606],
       [ 9.49283024,  6.1422784 , -0.97484846,  5.8604399 , -7.61126963,
         2.86555735, -7.25390288,  8.89609285,  0.33510318, -1.79181328,
        -4.66192239,  5.43323887, -0.86162507,  1.3705568 , -9.7904172 ,
         2.3613231 ,  2.20516237,  2.20604823,  8.76464833,  3.47795068],
       [-2.67206588, -1.30103177,  3.98418492, -8.88040428,  3.27735964,
         3.51616445, -5.81395151, -7.42287114, -3.73476887, -2.89520363,
         1.49435043, -1.35811028,  9.91250767, -7.86133474, -5.78975793,
        -6.54897163,  3.08083281, -5.18975209, -0.85563107, -5.06615534],
       [-6.85980599, -7.87144648,  3.33572279, -7.00394241, -5.97224874,
        -2.55638942,  6.36329802, -7.97988653,  

The small dataset will be the template for our large random dataset.
We'll use `dask.delayed` to adapt `sklearn.datasets.make_blobs`, so that the actual dataset is being generated on our workers. 

In [244]:
delayeds = [dask.delayed(make_blobs)(n_samples=n_samples_per_block,
                                     centers=centers,
                                     n_features=n_features,
                                     random_state=i)[0]
            for i in range(n_blocks)]
arrays = [da.from_delayed(obj, shape=(n_samples_per_block, n_features), dtype='float64')
          for obj in delayeds]
X = da.concatenate(arrays)
X

dask.array<concatenate, shape=(100000000, 20), dtype=float64, chunksize=(200000, 20)>

In [245]:
# Check the size of the array
X.nbytes / 1e9

16.0

In [246]:
# Only run this on the cluster.
X = X.persist()  

The algorithms implemented in Dask-ML are scalable. They handle larger-than-memory datasets just fine.

They follow the scikit-learn API, so if you're familiar with scikit-learn, you'll feel at home with Dask-ML.

In [251]:
from dask_ml.cluster import KMeans
clf = KMeans(init_max_iter=3, oversampling_factor=10)

In [252]:
%time clf.fit(X)

CPU times: user 15.5 s, sys: 615 ms, total: 16.1 s
Wall time: 41.7 s


KMeans(algorithm='full', copy_x=True, init='k-means||', init_max_iter=3,
       max_iter=300, n_clusters=8, n_jobs=1, oversampling_factor=10,
       precompute_distances='auto', random_state=None, tol=0.0001)

In [249]:
clf.labels_

dask.array<astype, shape=(100000000,), dtype=int32, chunksize=(200000,)>

In [250]:
clf.labels_[:10].compute()

array([1, 4, 1, 7, 7, 7, 1, 4, 6, 2], dtype=int32)

## Cleanup
To clean up, cancel the job that runs the cluster. 

In [253]:
run.cancel()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
